In [1]:
import os

In [2]:
os.getcwd()

'c:\\gitrepository\\Projects-AIML\\CVProject-Chest-Disease-Classification\\experiments'

In [3]:
os.chdir('../')
os.getcwd()

'c:\\gitrepository\\Projects-AIML\\CVProject-Chest-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_directory: Path
    source_url: str
    downloaded_data_file: Path
    unzip_directory: Path

In [5]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories(directory_list=[self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion
        create_directories(directory_list = [config.root_directory])

        data_ingestion_config = DataIngestionConfig(
            root_directory=config.root_directory,
            source_url= config.source_url,
            downloaded_data_file= config.downloaded_data_file,
            unzip_directory = config.unzip_directory
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from src.cnn_classifier.logger import logging
from src.cnn_classifier.utils.common import get_size
from src.cnn_classifier.entity.config_entity import DataIngestionConfig

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> str:
        """
        Code feteches the data and downloads from google drive
        """
        try:
            dataset_url = self.config.source_url
            zipped_file = self.config.downloaded_data_file
            os.makedirs(self.config.root_directory, exist_ok=True)
            file_id = dataset_url.split("/")[-2]
            logging.info(f"Started Downloading data from {dataset_url} into file {zipped_file}")
            url = f'https://drive.google.com/uc?id={file_id}'
            gdown.download(url, zipped_file)
            logging.info(f"Successfully downloaded data from {dataset_url} into file {zipped_file}")
        except Exception as e:
            raise e
        

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory and returns None
        """
        try:
            unzip_file_path = self.config.unzip_directory
            os.makedirs(unzip_file_path, exist_ok=True)
            with zipfile.ZipFile(self.config.downloaded_data_file, 'r') as zip_file:
                zip_file.extractall(unzip_file_path)
            logging.info(f"Unzipped file from {self.config.downloaded_data_file} to file {unzip_file_path}")
        except Exception as e:
            raise e

In [9]:
from src.cnn_classifier.config.configuration import ConfigurationManager
from src.cnn_classifier.components.data_ingestion import DataIngestion
from src.cnn_classifier.logger import logging

STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_data()
        data_ingestion.extract_zip_file()

In [10]:
try:
    logging.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
    obj = DataIngestionTrainingPipeline()
    obj.main()
    logging.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
    logging.exception(e)
    raise e

[2024-05-24 21:25:51,761: INFO: 259450960: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-05-24 21:25:51,785: INFO: common: YAML file - config\config.yaml loaded successfully]
[2024-05-24 21:25:51,802: INFO: common: YAML file - params.yaml loaded successfully]
[2024-05-24 21:25:51,804: INFO: common: created directory at: artifacts]
[2024-05-24 21:25:51,806: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-24 21:25:51,806: INFO: data_ingestion: Started Downloading data from https://drive.google.com/file/d/1oiTnXuWyL5oN6E4d-EmyplBj68ife5MJ/view?usp=drive_link into artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1oiTnXuWyL5oN6E4d-EmyplBj68ife5MJ
From (redirected): https://drive.google.com/uc?id=1oiTnXuWyL5oN6E4d-EmyplBj68ife5MJ&confirm=t&uuid=68c5d813-578b-4b99-8a4c-b0c434a643c9
To: c:\gitrepository\Projects-AIML\CVProject-Chest-Disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:14<00:00, 3.29MB/s]


[2024-05-24 21:26:09,271: INFO: data_ingestion: Successfully downloaded data from https://drive.google.com/file/d/1oiTnXuWyL5oN6E4d-EmyplBj68ife5MJ/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
[2024-05-24 21:26:10,890: INFO: data_ingestion: Unzipped file from artifacts/data_ingestion/data.zip to file artifacts/data_ingestion]
[2024-05-24 21:26:10,894: INFO: 259450960: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
